## TUSHAR NAGPAL -MT2022125
## MANDATE 3
### **In this mandate,**

#### As part of fine tuning:
#### --> I have used the concept of embedding and minimum distance calculation between them using two metrics:
####       1.1 Cosine Similarity
####       1.2 Eucledian Distance
       
#### --> I have removed the top most frequently occuring word from each IPC Section because it 
#### was impacting Cosine Similarity very much.
       
#### --> I have created whole dataset of Section-wise Indian Penal Codes.


### Future Work:

#### --> I will work on my dataset as it is not properly structured as it should be.
#### --> Will predict the context using this method which I have performed in this mandate and
#### pass this context to Pre-Trained model BERT for answering queries from that particular 
#### part of context.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ipc-section-wise/dataset_ipc.csv
/kaggle/input/ipc-section-wise/models.py


### Importing Neccessary Libraries

In [2]:
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
from scipy import spatial
import torch
import spacy
en_nlp = spacy.load('en_core_web_sm')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Loading the dataset for IPC which I have created using pdf available at https://www.indiacode.nic.in/
#### Dataset is made by separating each Indian Penal Code Section.

In [3]:
df = pd.read_csv('/kaggle/input/ipc-section-wise/dataset_ipc.csv',  encoding= 'unicode_escape')

In [4]:
df.drop(['Unnamed: 2'], axis = 1, inplace = True)

In [5]:
df

,ipc_sections,description
0,1,"""Title and extent of operation of the Code: Th..."
1,2,"""Punishment of offences committed within India..."
2,3,"""Punishment of offences committed beyond, but ..."
3,4,"""Extension of Code to extra-territorial offenc..."
4,5,"""Certain laws not to be affected by this Act: ..."
...,...,...
549,507,Criminal intimidation by an anonymous communic...
550,508,Act caused by inducing person to believe that ...
551,509,"Word, gesture or act intended to insult the mo..."
552,510,Misconduct in public by a drunken person.âWh...


### Data Preprocessing:

#### Removing stopwords, special characters and a top word which is most frequently occurring

In [6]:
import re
nltk.download('stopwords')
import gensim
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS

from collections import Counter

def preprocessingDesc(text):
#     text = re.sub(r"[^a-zA-Z0-9 ]", "", text)
    text = remove_stopwords(text)
    
    cnt = Counter()
    for word in text.split():
        cnt[word] += 1
    
    freqwords = set([w for(w,wc) in cnt.most_common(1)])
    text = ' '.join([word for word in str(text).split() if word not in freqwords])
    
    return text

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
df['description']=df['description'].map(lambda s:preprocessingDesc(s)) 

In [8]:
df['description'][131]

'Abetment soldier, sailor airman superior officer, execution office.â\x80\x94Whoever abets officer, soldier, 8 [sailor airman], Army, 9 [Navy Air Force] 10 [Government India], superior officer execution office, shall punished imprisonment description term extend years, shall liable fine.'

In [9]:
ipcs = df['description'].tolist()

In [10]:
# ipcs

### Downloading requirements to create Embeddings of each IPC section

In [11]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from random import randint

import numpy as np
import torch

In [12]:
mkdir dataset

mkdir: cannot create directory ‘dataset’: File exists


In [13]:
mkdir dataset/fastText

mkdir: cannot create directory ‘dataset/fastText’: File exists


In [14]:
!curl -Lo dataset/fastText/crawl-300d-2M.vec.zip https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1453M  100 1453M    0     0   249M      0  0:00:05  0:00:05 --:--:--  267M


In [15]:
# !unzip dataset/fastText/crawl-300d-2M.vec.zip -d dataset/fastText/

Archive:  dataset/fastText/crawl-300d-2M.vec.zip
replace dataset/fastText/crawl-300d-2M.vec? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [16]:
cd dataset

/kaggle/working/dataset


In [17]:
mkdir encoder

mkdir: cannot create directory ‘encoder’: File exists


In [18]:
!curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0   151M      0 --:--:-- --:--:-- --:--:--  151M


infersent is used for creating embedding and then cosine similarity will be computed between each ipc section and the question provided.

In [19]:
ls

encoder/  fastText/


In [20]:
cd ..

/kaggle/working


In [21]:
from shutil import copyfile
copyfile(src = "../input/ipc-section-wise/models.py", dst = "../working/models.py")

'../working/models.py'

importing models.py script from input to working directory

In [22]:
from models import InferSent
V = 2
MODEL_PATH = 'dataset/encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [23]:
W2V_PATH = 'dataset/fastText/crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)

In [24]:
infersent.build_vocab(ipcs, tokenize=True)

Found 3002(/3721) words with w2v vectors
Vocab size : 3002


In [25]:
from tqdm import tqdm

dict_embeddings = {}
# for i in range(len(ipcs)):
for i in tqdm(range(len(ipcs))):
#     print(i)
    dict_embeddings[ipcs[i]] = infersent.encode([ipcs[i]], tokenize=True)

100%|██████████| 554/554 [05:02<00:00,  1.83it/s]


#### dictionary embedding creation for each ipc section

In [90]:
questions = ["Punishment to family members if woman died within seven years of marriage with injury or burn ?"]
for i in range(len(questions)):
    questions[i] = questions[i]
df2 = pd.DataFrame(questions)
df2

,0
0,Punishment to family members if woman died wit...


#### Now, we will create embedding for question so that we can compare the question with all ipc sections using cosine similarity or euclidean distance.

In [91]:
df2.rename(columns={0: 'questions'}, inplace = True)

In [92]:
df2

,questions
0,Punishment to family members if woman died wit...


In [93]:
infersent.build_vocab(questions, tokenize=True)

Found 19(/19) words with w2v vectors
Vocab size : 19


In [94]:
for i in range(len(questions)):
    print(i)
    dict_embeddings[questions[i]] = infersent.encode([questions[i]], tokenize=True)

0


In [95]:
df2["ipcs"] = ['ipc']

In [96]:
df2["ipcs"][0] = ipcs
df2

,questions,ipcs
0,Punishment to family members if woman died wit...,"[""Title extent operation Code: This Act called..."


In [97]:
def process_data(df,df2):
    
    print("step 1")
    df2['sent_emb'] = df2['ipcs'].apply(lambda x: [dict_embeddings[item][0] if item in\
                                                           dict_embeddings else np.zeros(4096) for item in x])
    print("step 2")
    df2['quest_emb'] = df2['questions'].apply(lambda x: dict_embeddings[x] if x in dict_embeddings else np.zeros(4096) )
        
    return df2

In [98]:
df2 = process_data(df,df2)
df2

step 1
step 2


,questions,ipcs,sent_emb,quest_emb
0,Punishment to family members if woman died wit...,"[""Title extent operation Code: This Act called...","[[0.007468892, -0.06260717, 0.12939282, 0.0047...","[[0.007468892, -0.054872543, 0.09979467, -0.02..."


In [99]:
def cosine_sim(x):
    li = []
    for item in x["sent_emb"]:
        li.append(spatial.distance.cosine(item,x["quest_emb"][0]))
    return li

#### Function for calculating cosine similarity between question - ipc_section pair

In [100]:
def pred_idx(distances):
    return np.argmin(distances)

Taking index of ipc_section with minimum distance

In [101]:
def predictions(train):
    print(0)
    train["cosine_sim"] = train.apply(cosine_sim, axis = 1)
    print(1)
    train["diff"] = (train["quest_emb"] - train["sent_emb"])**2
    train["euclidean_dis"] = train["diff"].apply(lambda x: list(np.sum(x, axis = 1)))
    del train["diff"]
    
    print("cosine start")
    
    train["pred_idx_cos"] = train["cosine_sim"].apply(lambda x: pred_idx(x))
    train["pred_idx_euc"] = train["euclidean_dis"].apply(lambda x: pred_idx(x))
    
    return train

In [102]:
df2 = predictions(df2)

0
1
cosine start


In [103]:
df2 = df2.reset_index(drop=True)
df2

,questions,ipcs,sent_emb,quest_emb,cosine_sim,euclidean_dis,pred_idx_cos,pred_idx_euc
0,Punishment to family members if woman died wit...,"[""Title extent operation Code: This Act called...","[[0.007468892, -0.06260717, 0.12939282, 0.0047...","[[0.007468892, -0.054872543, 0.09979467, -0.02...","[0.6730236411094666, 0.5028181374073029, 0.491...","[8.951457, 6.736125, 6.3618727, 7.4799547, 7.7...",323,331


In [104]:
def showqa(train):
    for i in range(1):
        print("Question : " + train["questions"][i])
        print("Predicted Context : " + ipcs[train["pred_idx_cos"][i]])
        print("\n")

In [105]:
showqa(df2)

Question : Punishment to family members if woman died within seven years of marriage with injury or burn ?
Predicted Context : Dowry death.â(1) Where death woman caused burns bodily injury occurs normal circumstances seven years marriage shown soon death subjected cruelty harassment husband relative husband for, connection with, demand dowry, death called âdowry deathâ, husband relative deemed caused death. Explanation.âFor purposes sub-section, âdowryâ meaning section 2 Dowry Prohibition Act, 1961 (28 1961). (2) Whoever commits dowry death punished imprisonment term seven years extend imprisonment life.]




#### This is predicted context for our question where Context is an IPC_section